# **VGG16: Image Modality**
Se usa la arquitectura de redes neuronales convolucionales desarrollada por la Universidad de Oxford.

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd
from PIL import Image
import os

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### **Preprocesamiento de imágenes**
- Redimensionar y normalizar (224x224)
- Valores de los pixeles entre 0 y 1
- Normalizar los canales de color con respecto al dataset ImageNet

In [24]:
# Los datos ya fueron preprocesados y guardados como archivos .npy
# Falta cargar los datos
default_path = "C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/"
df = pd.read_csv("C:/Users/isaja/Documents/dcc/e/crisis/data/crisis_images_dataset.csv")

image_names = df['image_name'].tolist()
labels = df['image_info'].tolist()

In [25]:
image_paths = [default_path + name + '.npy' for name in image_names]
image_paths[:5]

['C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/917791044158185473_0.npy',
 'C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/917791130590183424_0.npy',
 'C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/917791291823591425_0.npy',
 'C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/917791291823591425_1.npy',
 'C:/Users/isaja/Documents/dcc/e/crisis/data/preprocessed_images/917792092100988929_0.npy']

In [33]:
image_path = image_paths[0]
if not os.path.exists(image_path):
                print(f"Archivo no encontrado: {image_path}")

In [34]:
def load_images_from_paths(image_paths, target_size=(224, 224)):
    """
    Carga imágenes desde rutas de archivo
    """
    images = []
    valid_indices = []
    
    print(f"Cargando {len(image_paths)} imágenes desde rutas...")
    
    for i, npy_path in enumerate(image_paths):
        try:
            # Verificar si el archivo existe
            if not os.path.exists(npy_path):
                print(f"Archivo no encontrado: {npy_path}")
                continue
                
            # Cargar array numpy
            img_array = np.load(npy_path)
            
            images.append(img_array)
            valid_indices.append(i)
            
        except Exception as e:
            print(f"Error cargando imagen {npy_path}: {e}")
            print(f"Forma del array: {img_array.shape}" if 'img_array' in locals() else "No se pudo cargar el array.")
            continue
            
        # Mostrar progreso cada 1000 imágenes
        if (i + 1) % 1000 == 0:
            print(f"Procesadas {i + 1}/{len(image_paths)} imágenes")
    
    print(f"Imágenes cargadas exitosamente: {len(images)}/{len(image_paths)}")
    
    return np.array(images), valid_indices

In [35]:

def load_and_prepare_data():
    """
    Carga los arrays numpy preprocesados (.npy) y prepara para entrenamiento
    """
    # Cargar los arrays
    X_paths = np.array(image_paths)
    y_labels = np.array(labels)
    
    # Verificar formas
    print(f"Número de rutas de imágenes: {len(X_paths)}")
    print(f"Número de etiquetas: {len(y_labels)}")

    # Cargar imágenes desde las rutas
    X_images, valid_indices = load_images_from_paths(X_paths)

    # Filtrar etiquetas para mantener solo las correspondientes a imágenes cargadas exitosamente
    y_labels_filtered = y_labels[valid_indices]
    
    # Mapeo manual de labels a números
    # Según el paper: 0 = Not-informative, 1 = Informative
    label_mapping = {'not_informative': 0, 'informative': 1}
    
    # Convertir etiquetas de texto a numéricas
    y_numeric = np.array([label_mapping[label] for label in y_labels_filtered])
    
    print(f"Distribución de etiquetas:")
    print(f"Non-informative (0): {np.sum(y_numeric == 0)}")
    print(f"Informative (1): {np.sum(y_numeric == 1)}")

    # Se asegura de tener el formato correcto
    X_images = X_images.astype('float32')

    y_categorical = to_categorical(y_numeric, num_classes=2)

    print(f"Rango de valores en X: min {X_images.min():.3f}, max {X_images.max():.3f}")

    print(f"Forma final de X: {X_images.shape}")
    print(f"Forma final de y: {y_categorical.shape}")
    return X_images, y_categorical

def split_data(X, y, test_size=0.3, val_size=0.5):
    """
    Divide los datos en train, validation y test y convierte a arrays numpy
    """
    X = np.array(X)
    y = np.array(y)

    print(f"Forma de X antes de split: {X.shape}")
    print(f"Forma de y antes de split: {y.shape}")

    # Primera división: train vs (val + test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    
    # Segunda división: val vs test
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=val_size, random_state=42, stratify=y_temp
    )

    print(f"Formas después del split:")
    print(f"Forma de X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Forma de X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"Forma de X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

### **Arquitectura del modelo**
Se usa el modelo pre-entrenado VGG16 para aplicar transfer learning
- Se busca reutilizar los pesos
- Se reemplaza la última capa por _softmax_ 

In [36]:

def create_vgg16_model(num_classes=2, input_shape=(224, 224, 3)):
    """
    Crea el modelo VGG16 para clasificación binaria
    según la descripción del paper
    """
    # Cargar VGG16 pre-entrenado en ImageNet (sin la capa final)
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Congelar las capas convolucionales (transfer learning)
    for layer in base_model.layers:
        layer.trainable = False
    
    # Añadir capas personalizadas según el paper
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1000, activation='relu')(x)  # Capa fc2 mencionada en el paper
    x = BatchNormalization()(x)
    x = Dropout(0.02)(x)  # Dropout rate usado en el paper
    
    # Capa de salida
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Modelo final
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

### **Entrenamiento**
- Uso del Adam optimizer con initial learning rate de 10^-6
- Se reduce la tasa de aprendizaje en 0.1 si el accuracy deja de mejorar después de las 100 épocas
- El máximo de épocas es 1000, con early stopping

In [37]:
def setup_training(model):
    """
    Configura el optimizador y callbacks según el paper
    """
    # Optimizador Adam con learning rate muy bajo (1e-6) como en el paper
    optimizer = Adam(learning_rate=1e-6)
    
    # Callbacks
    callbacks = [
        # Early stopping con paciencia de 100 epochs (como en el paper)
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=100,
            restore_best_weights=True
        ),
        # ReduceLROnPlateau (mencionado en el paper)
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.1,
            patience=50,
            min_lr=1e-8
        ),
        # Model checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            'best_vgg16_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
    ]
    
    # Compilar modelo
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return callbacks

### **Evaluación del modelo**

In [38]:

def evaluate_model(model, X_test, y_test):
    """
    Evaluación completa del modelo
    """
    # Predicciones
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    
    # Classification report
    print("Classification Report:")
    print(classification_report(y_true_classes, y_pred_classes, 
                              target_names=['Not-informative', 'Informative']))
    
    # Matriz de confusión
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not-inf', 'Inf'],
                yticklabels=['Not-inf', 'Inf'])
    plt.title('Matriz de Confusión - VGG16')
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.show()
    
    return y_pred_classes, y_true_classes

### **Experimento**

In [39]:
def main():
    # 1. Cargar y preparar datos
    print("Cargando datos...")
    X, y = load_and_prepare_data()
    
    # 2. Dividir datos
    print("Dividiendo datos...")
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)
    
    print(f"Train: {X_train.shape[0]} samples")
    print(f"Validation: {X_val.shape[0]} samples") 
    print(f"Test: {X_test.shape[0]} samples")
    
    # 3. Crear modelo
    print("Creando modelo VGG16...")
    model = create_vgg16_model(num_classes=2)
    
    # 4. Configurar entrenamiento
    callbacks = setup_training(model)
    
    # 5. Entrenar modelo
    print("Iniciando entrenamiento...")
    history = model.fit(
        X_train, y_train,
        batch_size=32,  # Minibatch size del paper
        epochs=1000,    # Máximo de epochs del paper
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    
    # 6. Evaluar en test
    print("Evaluando en test set...")
    test_results = model.evaluate(X_test, y_test, verbose=0)
    
    print("\n=== RESULTADOS FINALES ===")
    print(f"Test Loss: {test_results[0]:.4f}")
    print(f"Test Accuracy: {test_results[1]:.4f}")
    print(f"Test Precision: {test_results[2]:.4f}")
    print(f"Test Recall: {test_results[3]:.4f}")
    
    # Calcular F1-score
    precision = test_results[2]
    recall = test_results[3]
    f1 = 2 * (precision * recall) / (precision + recall)
    print(f"Test F1-score: {f1:.4f}")
    
    return model, history, test_results

# Ejecutar el entrenamiento
if __name__ == "__main__":
    model, history, results = main()
    evaluate_model(model, X_test, y_test)

Cargando datos...
Número de rutas de imágenes: 18082
Número de etiquetas: 18082
Cargando 18082 imágenes desde rutas...
Procesadas 1000/18082 imágenes
Procesadas 2000/18082 imágenes
Procesadas 3000/18082 imágenes
Procesadas 4000/18082 imágenes
Procesadas 5000/18082 imágenes
Procesadas 6000/18082 imágenes
Procesadas 7000/18082 imágenes
Procesadas 8000/18082 imágenes
Procesadas 9000/18082 imágenes
Procesadas 10000/18082 imágenes
Procesadas 11000/18082 imágenes
Procesadas 12000/18082 imágenes
Procesadas 13000/18082 imágenes
Procesadas 14000/18082 imágenes
Procesadas 15000/18082 imágenes
Procesadas 16000/18082 imágenes
Procesadas 17000/18082 imágenes
Procesadas 18000/18082 imágenes
Imágenes cargadas exitosamente: 18082/18082
Distribución de etiquetas:
Non-informative (0): 8708
Informative (1): 9374
Rango de valores en X: min -2.118, max 2.640
Forma final de X: (18082, 224, 224, 3)
Forma final de y: (18082, 2)
Dividiendo datos...
Forma de X antes de split: (18082, 224, 224, 3)
Forma de y ant

KeyboardInterrupt: 

In [ ]:
# Paso 1: Solo diagnóstico
def diagnostic_check():
    print("=== DIAGNÓSTICO COMPLETO ===")
    
    # Cargar datos
    X_images = np.array(image_paths)
    y_labels = np.array(labels)
    
    print("1. DATOS ORIGINALES:")
    print(f"X_images - Tipo: {X_images.dtype}, Forma: {X_images.shape}")
    print(f"y_labels - Tipo: {y_labels.dtype}, Valores únicos: {np.unique(y_labels)}")
    
    return X_images, y_labels

# Ejecutar solo el diagnóstico primero
X_orig, y_orig = diagnostic_check()

=== DIAGNÓSTICO COMPLETO ===
1. DATOS ORIGINALES:
X_images - Tipo: <U87, Forma: (18082,)
y_labels - Tipo: <U15, Valores únicos: ['informative' 'not_informative']
